In [2]:
# regression analysis of what plays when it comes to emotions and engagement metrics and how they correlate with each other

In [3]:
# import libraries
import pandas as pd

from spark_session_manager import SparkSessionManager
from pyspark.sql import functions as F


In [8]:
spark =  SparkSessionManager().get_spark_session()

24/04/22 14:23:24 WARN Utils: Your hostname, gr00stl-Legion-Y540-15IRH-PG0 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp0s20f3)
24/04/22 14:23:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/22 14:23:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
# load data (parquet)
df = spark.read.parquet("data/processed/tranformed_US_youtube_trending_data_min.parquet")

# show head
df.show(5)

# show schema
df.printSchema()

+----------+-----------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------------+------------------+--------+------------------+--------------------+-----------------+----------------+--------------------+--------------+
|categoryId|   video_id|               title|         publishedAt|           channelId|  channelTitle|       trending_date|                tags|        view_count|             likes|dislikes|     comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         description| categoryTitle|
+----------+-----------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------------+------------------+--------+------------------+--------------------+-----------------+----------------+--------------------+--------------+
|        20|_I9T4hxmZXU|Lunar New Year Ma...|2023-01-18T07:59:02Z|UCD1Em4q90ZUK2R5H...|Clas

In [ ]:
# build base model

In [ ]:
## evaluate model

In [ ]:
# build imporved model (maybe with interactions?)